# Считывание данных из Google Sheet

Набор данных оформлен [в виде электронной таблицы на Google Drive](https://docs.google.com/spreadsheets/d/1KSirtO9hZSmVst--GiqsBPYk6hX-xOCI-SolgiafjcI/edit?usp=sharing). Доступ к Google Sheets API осуществляется в соответствии с [документацией API для Python](https://developers.google.com/sheets/api/quickstart/python).

Импортируем данные учетной записи и преобразуем в объект Pandas:

In [17]:
import collections
from datetime import datetime
from datetime import timedelta
import pandas as pd

In [5]:
df = pd.read_csv('data.csv', index_col=0)

In [9]:
not_published = df[253:]

In [16]:
rus = '{0}:\n\n«{1}».\n\n#воспоминание {2}, #непереводное\n'
eng = '{0}:\n\n"{1}".\n\n#memory {2}, #translated #rus\n'

for i, item in enumerate(not_published.index):
    name_rus = df.loc[item]['Имя']
    if name_rus == '-':
        name_rus = 'Без имени'
#     name_eng = df.loc[str(item)]['Name']
#     if name_eng == '-':
#         name_eng = 'No name'
    
    memory_rus = df.loc[item]['Воспоминание'].strip()
    if memory_rus[-1] == '.':
        memory_rus = memory_rus[:-1]
        
#     memory_eng = df.loc[str(item)]['Translation'].strip()
#     if memory_eng[-1] == '.':
#         memory_eng = memory_eng[:-1]
    
    d = (datetime.today()+timedelta(days=2)+timedelta(days=i)).date()
    
    print("===== {} ====".format(d))
    print(rus.format(name_rus, memory_rus, item))
    #print(eng.format(name_eng, memory_eng, item))

===== 2020-08-25 ====
Настя:

«Я помню как мама поет мне колыбельную. Лежу в кроватке, вижу ее деревянные прутья. Маму я не вижу, только слышу. А еще через прутья видна кухня и кухонное окно. А в окне то ли рассвет, то ли закат. Сумерки, и небо алеет. Мне больше двух лет. Мамина песня грустная».

#воспоминание 254, #непереводное

===== 2020-08-26 ====
Олег:

«Я маленький танцую и бешусь под песню Олега Газманова, концерт которого показывают по телевизору. Вокруг много взрослых. Мне показалось странным, что первое воспоминание связано с таким взрослым возрастом. По моим ощущением три-четыре года, то есть я разговариваю. Есть подозрение, что я видел кассету с видео того как я это делаю».

#воспоминание 255, #непереводное

===== 2020-08-27 ====
Карина:

«Зима. Мама обула меня в валенки. Мне очень неудобно, они колючие и твердые и я не могу устоять в них, мне ужасно горько из-за этого. Я не понимаю почему у меня такая неудобная обувь. Я тогда только научилась ходить.

Весна, вечер. Мне пят